## Data Preprocessing

In [50]:
data= pd.read_csv('gram_df.csv')
data.head()

<IPython.core.display.Javascript object>

,district,market,commodity,variety,price,day,month,year
0,Hassan,Arasikere,Tomato,Tomato,2000.0,8.0,Mar,2017.0
1,Hassan,Arasikere,Tomato,Tomato,1000.0,2.0,Aug,2016.0
2,Hassan,Arasikere,Tomato,Tomato,1000.0,30.0,Jul,2016.0
3,Hassan,Arasikere,Tomato,Tomato,1000.0,26.0,Jul,2016.0
4,Hassan,Arasikere,Tomato,Tomato,1000.0,23.0,Jul,2016.0


In [51]:
from sklearn.preprocessing import LabelEncoder     # applying albel encoder for labelling categorical variables
LE= LabelEncoder()

In [52]:
data.district= LE.fit_transform(data.district)
data.market= LE.fit_transform(data.market)
data.variety= LE.fit_transform(data.variety)
data.month= LE.fit_transform(data.month)
data.head()

,district,market,commodity,variety,price,day,month,year
0,9,0,Tomato,1,2000.0,8.0,7,2017.0
1,9,0,Tomato,1,1000.0,2.0,1,2016.0
2,9,0,Tomato,1,1000.0,30.0,5,2016.0
3,9,0,Tomato,1,1000.0,26.0,5,2016.0
4,9,0,Tomato,1,1000.0,23.0,5,2016.0


In [53]:
data.district.unique()

array([ 9,  4, 11,  1,  2,  0,  3,  6,  7,  8, 12, 10,  5, 15, 13, 14, 17,
       16, 18, 19])

In [73]:
data[['district','variety','market','day','month','year','price']]=data[['district',
                                                                 'variety','market','day',       #changing the datatypes to integers
                                                                 'month','year','price']].astype(float)

In [74]:
data.dtypes

district    float64
market      float64
variety     float64
price       float64
day         float64
month       float64
year        float64
dtype: object

In [75]:
data=data.drop('commodity',axis=1)
data.head()

KeyError: "['commodity'] not found in axis"

In [56]:
data.market.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58])

## Feature selection 

In [76]:
x= data.drop('price',axis=1)
y= data.price

In [77]:
x.head()

,district,market,variety,day,month,year
0,9.0,0.0,1.0,8.0,7.0,2017.0
1,9.0,0.0,1.0,2.0,1.0,2016.0
2,9.0,0.0,1.0,30.0,5.0,2016.0
3,9.0,0.0,1.0,26.0,5.0,2016.0
4,9.0,0.0,1.0,23.0,5.0,2016.0


In [78]:
y.head()

0    2000.0
1    1000.0
2    1000.0
3    1000.0
4    1000.0
Name: price, dtype: float64

In [79]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)  #splitting the data
print('x_train=',x_train.shape)
print('x_test=',x_test.shape)
print('y_train=',y_train.shape)
print('y_test=',y_test.shape)

x_train= (23742, 6)
x_test= (11695, 6)
y_train= (23742,)
y_test= (11695,)


In [80]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel     #algorithm for feat selection
select=SelectFromModel(Lasso(alpha= 0.05))
select.fit(x_train,y_train)

SelectFromModel(estimator=Lasso(alpha=0.05))

In [81]:
select.get_support()              # all features turned out to be important 

array([ True,  True,  True,  True,  True,  True])

## Modelling

In [19]:
from sklearn.model_selection import RandomizedSearchCV
#Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [21]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 5, verbose=2, random_state=42, n_jobs = 1)
rf_random.fit(x_train,y_train)

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] n_estimators=400, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=5 
[CV]  n_estimators=400, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=5, total=   3.0s
[CV] n_estimators=400, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.9s remaining:    0.0s


[CV]  n_estimators=400, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=5, total=   3.0s
[CV] n_estimators=400, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=5 
[CV]  n_estimators=400, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=5, total=   3.1s
[CV] n_estimators=400, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=5 
[CV]  n_estimators=400, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=5, total=   3.0s
[CV] n_estimators=400, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=5 
[CV]  n_estimators=400, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=5, total=   3.0s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=20 
[CV]  n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=20, total=   8.3s
[CV] n_estimators=200, min_samples_spl

[CV]  n_estimators=1000, min_samples_split=15, min_samples_leaf=10, max_features=sqrt, max_depth=10, total=  11.5s
[CV] n_estimators=1000, min_samples_split=15, min_samples_leaf=10, max_features=sqrt, max_depth=10 
[CV]  n_estimators=1000, min_samples_split=15, min_samples_leaf=10, max_features=sqrt, max_depth=10, total=  11.5s
[CV] n_estimators=1000, min_samples_split=15, min_samples_leaf=10, max_features=sqrt, max_depth=10 
[CV]  n_estimators=1000, min_samples_split=15, min_samples_leaf=10, max_features=sqrt, max_depth=10, total=  11.6s
[CV] n_estimators=1000, min_samples_split=15, min_samples_leaf=10, max_features=sqrt, max_depth=10 
[CV]  n_estimators=1000, min_samples_split=15, min_samples_leaf=10, max_features=sqrt, max_depth=10, total=  11.5s
[CV] n_estimators=1000, min_samples_split=15, min_samples_leaf=10, max_features=sqrt, max_depth=10 
[CV]  n_estimators=1000, min_samples_split=15, min_samples_leaf=10, max_features=sqrt, max_depth=10, total=  11.6s
[CV] n_estimators=100, mi

...

[CV]  n_estimators=500, min_samples_split=2, min_samples_leaf=5, max_features=auto, max_depth=20, total=  17.2s
[CV] n_estimators=500, min_samples_split=2, min_samples_leaf=5, max_features=auto, max_depth=20 
[CV]  n_estimators=500, min_samples_split=2, min_samples_leaf=5, max_features=auto, max_depth=20, total=  21.7s
[CV] n_estimators=900, min_samples_split=10, min_samples_leaf=10, max_features=sqrt, max_depth=25 
[CV]  n_estimators=900, min_samples_split=10, min_samples_leaf=10, max_features=sqrt, max_depth=25, total=  15.7s
[CV] n_estimators=900, min_samples_split=10, min_samples_leaf=10, max_features=sqrt, max_depth=25 
[CV]  n_estimators=900, min_samples_split=10, min_samples_leaf=10, max_features=sqrt, max_depth=25, total=  14.6s
[CV] n_estimators=900, min_samples_split=10, min_samples_leaf=10, max_features=sqrt, max_depth=25 
[CV]  n_estimators=900, min_samples_split=10, min_samples_leaf=10, max_features=sqrt, max_depth=25, total=  14.2s
[CV] n_estimators=900, min_samples_split

[CV]  n_estimators=1200, min_samples_split=2, min_samples_leaf=5, max_features=sqrt, max_depth=30, total=  12.1s
[CV] n_estimators=1200, min_samples_split=2, min_samples_leaf=5, max_features=sqrt, max_depth=30 
[CV]  n_estimators=1200, min_samples_split=2, min_samples_leaf=5, max_features=sqrt, max_depth=30, total=  12.1s
[CV] n_estimators=1200, min_samples_split=2, min_samples_leaf=5, max_features=sqrt, max_depth=30 
[CV]  n_estimators=1200, min_samples_split=2, min_samples_leaf=5, max_features=sqrt, max_depth=30, total=  12.1s
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=30 
[CV]  n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=30, total=   6.7s
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=30 
[CV]  n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=30, total=   6.7s
[CV] n_estimators=600, min_samples_split=10

[CV]  n_estimators=900, min_samples_split=100, min_samples_leaf=1, max_features=sqrt, max_depth=5, total=   4.0s
[CV] n_estimators=900, min_samples_split=100, min_samples_leaf=1, max_features=sqrt, max_depth=5 
[CV]  n_estimators=900, min_samples_split=100, min_samples_leaf=1, max_features=sqrt, max_depth=5, total=   4.0s
[CV] n_estimators=900, min_samples_split=100, min_samples_leaf=1, max_features=sqrt, max_depth=5 
[CV]  n_estimators=900, min_samples_split=100, min_samples_leaf=1, max_features=sqrt, max_depth=5, total=   4.0s
[CV] n_estimators=900, min_samples_split=100, min_samples_leaf=1, max_features=sqrt, max_depth=5 
[CV]  n_estimators=900, min_samples_split=100, min_samples_leaf=1, max_features=sqrt, max_depth=5, total=   4.0s
[CV] n_estimators=900, min_samples_split=100, min_samples_leaf=1, max_features=sqrt, max_depth=30 
[CV]  n_estimators=900, min_samples_split=100, min_samples_leaf=1, max_features=sqrt, max_depth=30, total=   6.7s
[CV] n_estimators=900, min_samples_split=

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed: 44.1min finished


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=50, n_jobs=1,
                   param_distributions={'max_depth': [5, 10, 15, 20, 25, 30],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 5, 10],
                                        'min_samples_split': [2, 5, 10, 15,
                                                              100],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000, 1100,
                                                         1200]},
                   random_state=42, verbose=2)

In [22]:
rf_random.score(x_test,y_test)

0.8664469240978179

In [82]:
rf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 20}

In [83]:
model=RandomForestRegressor(n_estimators= 200,
 min_samples_split= 5,
 min_samples_leaf= 1,
 max_features= 'auto',
 max_depth= 20)

In [84]:
model.fit(x_train,y_train)

RandomForestRegressor(max_depth=20, min_samples_split=5, n_estimators=200)

In [85]:
model.score(x_test,y_test)

0.8661173187616933

In [38]:
import pickle
file= open('tomato.pkl','wb')
pickle.dump(model, file)

In [35]:
file= open('assngmt.pkl','wb')
pickle.dump(rf_random, file)

In [86]:
import pickle
name='gram_model.sav'
pickle.dump(model, open(name,'wb'))

In [87]:
load_model= pickle.load(open(name,'rb'))

In [88]:
load_model.predict(x_test)

array([ 868.71946212,  702.41989705,  470.6125657 , ..., 2216.68383622,
       1901.46909127,  965.73930425])

In [89]:
load_model.predict([[9,0,1,2,1,2016
                    ]])

array([1443.80390079])

In [93]:
x_test.to_csv('gram_test.csv',index=False)

In [94]:
x_test

,district,market,variety,day,month,year
25882,11.0,41.0,1.0,19.0,9.0,2018.0
13793,6.0,13.0,1.0,21.0,11.0,2017.0
13481,6.0,13.0,1.0,26.0,3.0,2019.0
29892,0.0,45.0,1.0,21.0,8.0,2020.0
19314,2.0,27.0,1.0,18.0,9.0,2018.0
...,...,...,...,...,...,...
17461,9.0,20.0,1.0,31.0,2.0,2020.0
27599,15.0,42.0,1.0,16.0,2.0,2019.0
14156,6.0,13.0,1.0,6.0,4.0,2016.0
24821,11.0,39.0,1.0,16.0,5.0,2016.0
